In [1]:
from tensorflow import keras
from tensorflow.keras import regularizers
from random import shuffle

import numpy as np
import cv2 as cv
import glob
import os

In [2]:
BATCH_SIZE = 32     # number of images to be processed in a batch
NUM_EPOCHS = 400     # number of times the entire dataset is passed through the network
NUM_CLASSES = 1     # number of classes in the dataset
t = 224             # target size of image

In [3]:
# Load the InceptionV3 model
base_model = keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=(t, t, 3))

# Freeze the pretrained weights
for layer in base_model.layers:
    layer.trainable = False

In [4]:
# Added layer to base model
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
predictions = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(x)

In [5]:
# The model we will train
model = keras.models.Model(inputs=base_model.input, outputs=predictions)

#optimizer = keras.optimizers.Adam(learning_rate=0.001)      # Adam optimizer

initial_learning_rate = 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, 
    decay_steps=10000, 
    decay_rate=0.9, 
    staircase=True
)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

loos_fn = keras.losses.BinaryCrossentropy()                 # Binary crossentropy loss function
model.compile(optimizer=optimizer, loss=loos_fn, metrics=['accuracy'])  # Compile the model

In [6]:
# Load the dataset
x_train = []
y_train = []
x_test = []
y_test = []

In [7]:
#tipo = "mirror"
tipo = "black"

In [8]:
# Load the training data
dataTr = []

for filename in glob.glob(os.path.join("D:/SkinCancerDatasets/dataset/"+tipo+"/train/bcc/",'*.jpg')):
    dataTr.append([1, cv.resize(cv.imread(filename), dsize=(t, t), interpolation=cv.INTER_CUBIC)])
for filename in glob.glob(os.path.join("D:/SkinCancerDatasets/dataset/"+tipo+"/train/scc/",'*.jpg')):
    dataTr.append([0, cv.resize(cv.imread(filename), dsize=(t, t), interpolation=cv.INTER_CUBIC)])

print('Images loaded', len(dataTr))

Images loaded 6518


In [9]:
# Shuffle data
for i, j in dataTr:
    x_train.append(j)
    y_train.append(i)
x_train = np.array(x_train)/255 # Normalize
y_train = np.array(y_train)

In [10]:
for filename in glob.glob(os.path.join("D:/SkinCancerDatasets/dataset/"+tipo+"/validation/bcc/",'*.jpg')):
    x_test.append(cv.resize(cv.imread(filename), dsize=(t, t), interpolation=cv.INTER_CUBIC))
    y_test.append(1)
for filename in glob.glob(os.path.join("D:/SkinCancerDatasets/dataset/"+tipo+"/validation/scc/",'*.jpg')):
    x_test.append(cv.resize(cv.imread(filename), dsize=(t, t), interpolation=cv.INTER_CUBIC))
    y_test.append(0)

print('Images loaded', len(x_test))
print('Labels loaded', len(y_test))

Images loaded 2794
Labels loaded 2794


In [11]:
x_test = np.array(x_test)/255 # Normalize
y_test = np.array(y_test)

In [12]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 100,
    verbose = 0,
    restore_best_weights = True
)

In [13]:
checkpoint_filepath = 'best_cnn1_MobilNet.h5'

model_checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

In [14]:
history = model.fit(
    x_train, y_train,
    batch_size = BATCH_SIZE,
    epochs = NUM_EPOCHS,
    callbacks = [early_stopping, model_checkpoint],
    validation_data = (x_test, y_test)
)

Epoch 1/400
204/204 [==============================] - ETA: 0s - loss: 2.7581 - accuracy: 0.7284
Epoch 1: val_loss improved from inf to 0.88717, saving model to best_cnn1_MobilNet.h5


c:\Users\josei\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


204/204 [==============================] - 137s 660ms/step - loss: 2.7581 - accuracy: 0.7284 - val_loss: 0.8872 - val_accuracy: 0.7792
Epoch 2/400
204/204 [==============================] - ETA: 0s - loss: 0.7402 - accuracy: 0.7518
Epoch 2: val_loss improved from 0.88717 to 0.60147, saving model to best_cnn1_MobilNet.h5
204/204 [==============================] - 123s 603ms/step - loss: 0.7402 - accuracy: 0.7518 - val_loss: 0.6015 - val_accuracy: 0.7989
Epoch 3/400
204/204 [==============================] - ETA: 0s - loss: 0.6136 - accuracy: 0.7579
Epoch 3: val_loss improved from 0.60147 to 0.57137, saving model to best_cnn1_MobilNet.h5
204/204 [==============================] - 123s 605ms/step - loss: 0.6136 - accuracy: 0.7579 - val_loss: 0.5714 - val_accuracy: 0.7724
Epoch 4/400
204/204 [==============================] - ETA: 0s - loss: 0.5687 - accuracy: 0.7620
Epoch 4: val_loss did not improve from 0.57137
204/204 [==============================] - 122s 599ms/step - loss: 0.5687 - a

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('Test loss: {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_acc*100.))

In [ ]:
model.save('cnn1_MobilNet.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Cargar el mejor modelo guardado
best_model = load_model('best_cnn1_MobilNet.h5')

# Hacer predicciones con el mejor modelo
test_loss, test_acc = best_model.evaluate(x_test, y_test, verbose=2)
print('Test loss: {0: .4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_acc*100.))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Supongamos que tienes un historial llamado 'history' con registros de pérdida y precisión para entrenamiento y validación.
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(train_loss) + 1)

# Crear subplots
fig, axes = plt.subplots(1, 2, figsize=(10, 3))

# Gráfico para pérdida
plt.sca(axes[0])
sns.lineplot(x=epochs, y=train_loss, label='Train Loss')
sns.lineplot(x=epochs, y=val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Gráfico para precisión
plt.sca(axes[1])
sns.lineplot(x=epochs, y=train_accuracy, label='Train Accuracy')
sns.lineplot(x=epochs, y=val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Ajusta la disposición
plt.tight_layout()

# Muestra la figura
plt.show()


In [ ]:
y_pred = model.predict(x_test)
y_pred = np.round(y_pred).astype(int)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Supongamos que tienes las etiquetas reales (y_true) y las predicciones (y_pred).
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)
print(cr)

class_labels = ["bcc", "scc"]

# Crear un mapa de calor para la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Supongamos que tienes etiquetas reales (y_true) y probabilidades de predicción (y_probs).
fpr, tpr, _ = roc_curve(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Crear un gráfico de la curva ROC
plt.figure(figsize=(8, 6))
sns.lineplot(x=fpr, y=tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

In [ ]:
output_directory = "D:/SkinCancerDatasets/dataset/errors/"+tipo+"/"
os.makedirs(output_directory, exist_ok=True)

for image, true_label, predicted_label in zip(x_test, y_test, y_pred):
    if true_label != predicted_label:
        if predicted_label == 1:
            label_name = "bcc"
        else:
            label_name = "scc"
    
        # Crea un directorio para la etiqueta si no existe
        label_directory = os.path.join(output_directory, label_name)
        os.makedirs(label_directory, exist_ok=True)
    
        # Genera un nombre de archivo único para la imagen
        image_name = f"image_{len(os.listdir(label_directory))}.jpg"
        output_path = os.path.join(label_directory, image_name)
    
        # Guarda la imagen en la carpeta de la etiqueta
        cv.imwrite(output_path, image * 255)

In [ ]:
monto_mensual = 20000
years = 10
interes_anual = 0.10

tiempo = years * 12
total = 0

for i in range(tiempo):
    total = total + monto_mensual
    total = total * (1 + interes_anual/12)

print(monto_mensual*tiempo)
print(total)